In [1]:
import sys
import pandas as pd
import numpy as np
import math
from scipy.special import softmax
from scipy.optimize import minimize, LinearConstraint
from scipy.spatial.distance import braycurtis
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from imports import *
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
import time
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import social_model_class_v2
reload(social_model_class_v2)
# reload(imports)


<module 'social_model_class_v2' from '/home/jupyter-tomeroron/baboons/social_model_class_v2.py'>

In [2]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"

In [3]:
all_model = social_model_class_v2.superModel(data_path, metadata_path)

np.all(all_model.metadata_df.index == all_model.data_df.index)

/home/jupyter-tomeroron/baboons/social_model_class_v2.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean['collection_date'] = pd.to_datetime(metadata_clean['collection_date'])


True

In [4]:
all_model.gamma_ =0.95717135

In [ ]:
# t = time.time()
# all_model.fit()
# print(time.time() - t)

## cross validation (4 - fold)

In [ ]:

baboons = all_model.baboons
dict_list = list(all_model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = all_model.data_df
metadata_df = all_model.metadata_df

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [ ]:

baboons = all_model.baboons
dict_list = list(all_model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]
metadata_df, data_df = social_model_class_v2.preprocessing(data_path, metadata_path)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    
for i in range(4):
    model = social_model_class_v2.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    futures =[]
    baboon_ids = []
    res = []
    # model.fit()
    model.gamma_ = all_model.gamma_
    with ProcessPoolExecutor(cpus) as executor:
    
        future_to_baboon = {executor.submit(baboon.fit, model.alpha_, model.gamma_): baboon for baboon in model.baboons.values()}
    
    for future in as_completed(future_to_baboon):
        baboon = future_to_baboon[future]  # Get the baboon that corresponds to this future
        baboon.beta_ = future.result()[0]  # Get the result of the computation
        # print(baboon.beta_)   
    # for baboon in model.baboons.values():
    #     baboon.beta_= np.random.rand(61)
    
    print(f"predicting {i}")
    
    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")

    print(f"new data added {i}")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.extend(score_noniter)
        bc_iterative.extend(score_iter)

    # Plot KDE plots for both noniterative and iterative scores
    sns.kdeplot(bc_noniterative, ax=axes[i, 0], label="noniterative")
    sns.kdeplot(bc_iterative, ax=axes[i, 1], label="iterative")
    axes[i, 0].set_xlim(0, 1)
    axes[i, 1].set_xlim(0, 1)
    axes[i, 0].vlines(np.mean(bc_noniterative), 0, 3.5, color='red', linestyle='--')
    axes[i, 1].vlines(np.mean(bc_iterative), 0, 3.5, color='red', linestyle='--')

    # Set titles
    axes[i, 0].set_title(f"Non-iterative (Group {i+1}), mean={np.mean(bc_noniterative):.3f}")
    axes[i, 1].set_title(f"Iterative (Group {i+1}), mean={np.mean(bc_iterative):.3f}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))
        
for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.extend(score_noniter)
        bc_iterative.extend(score_iter)


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[0], label=f"Group {i}, mean={np.mean(bc_noniterative):.3f}")
    sns.kdeplot(bc_iterative, ax=axes[1],  label=f"Group {i}, mean={np.mean(bc_iterative):.3f}")

# Set titles and labels
axes[0].set_title("Non-iterative")
axes[1].set_title("Iterative")
axes[0].set_xlabel("Bray-Curtis Distance")
axes[1].set_xlabel("Bray-Curtis Distance")
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()

### Only first 2

In [ ]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = pd.read_csv(data_path, index_col=0)
metadata_df = pd.read_csv(metadata_path, index_col=0)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:2])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [ ]:
baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]
metadata_df, data_df = social_model_class_v2.preprocessing(data_path, metadata_path)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:2])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    
for i in range(4):
    model = social_model_class_v2.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    model.gamma_ = all_model.gamma_
    
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    with ProcessPoolExecutor(cpus) as executor:
    
        future_to_baboon = {executor.submit(baboon.fit, model.alpha_, model.gamma_): baboon for baboon in model.baboons.values()}
    
    for future in as_completed(future_to_baboon):
        baboon = future_to_baboon[future]  # Get the baboon that corresponds to this future
        baboon.beta_ = future.result()[0]  # Get the result of the computation
        
    # for baboon in model.baboons.values():
    #     baboon.beta_= np.random.rand(61)
    
    print(f"predicting {i}")
    
    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")

    print(f"new data added {i}")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.extend(score_noniter)
        bc_iterative.extend(score_iter)


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[i, 0], label="noniterative")
    sns.kdeplot(bc_iterative, ax=axes[i, 1], label="iterative")
    axes[i, 0].set_xlim(0, 1)
    axes[i, 1].set_xlim(0, 1)
    axes[i, 0].vlines(np.mean(bc_noniterative), 0, 3.5, color='red', linestyle='--')
    axes[i, 1].vlines(np.mean(bc_iterative), 0, 3.5, color='red', linestyle='--')

    # Set titles
    axes[i, 0].set_title(f"Non-iterative (Group {i+1}) Super Short (first 2), mean={np.mean(bc_noniterative):.3f}")
    axes[i, 1].set_title(f"Iterative (Group {i+1}) Super Short (first 2), mean={np.mean(bc_iterative):.3f}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))

# Initialize lists to accumulate Bray-Curtis distances for all groups
bc_noniterative = []
bc_iterative = []

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.extend(score_noniter)
        bc_iterative.extend(score_iter)


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[0], label=f"Group {i}, mean={np.mean(bc_noniterative):.3f}")
    sns.kdeplot(bc_iterative, ax=axes[1],  label=f"Group {i}, mean={np.mean(bc_iterative):.3f}")

# Set titles and labels
axes[0].set_title("Non-iterative Super Short (first 2)")
axes[1].set_title("Iterative Super Short (first 2)")
axes[0].set_xlabel("Bray-Curtis Distance")
axes[1].set_xlabel("Bray-Curtis Distance")
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()

### all but last 10

In [ ]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = pd.read_csv(data_path, index_col=0)
metadata_df = pd.read_csv(metadata_path, index_col=0)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:-10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [ ]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]
metadata_df, data_df = social_model_class_v2.preprocessing(data_path, metadata_path)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:-10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    
for i in range(4):
    model = social_model_class_v2.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    model.gamma_ = all_model.gamma_
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    with ProcessPoolExecutor(cpus) as executor:
    
        future_to_baboon = {executor.submit(baboon.fit, model.alpha_, model.gamma_): baboon for baboon in model.baboons.values()}
    
    for future in as_completed(future_to_baboon):
        baboon = future_to_baboon[future]  # Get the baboon that corresponds to this future
        baboon.beta_ = future.result()[0]  # Get the result of the computation
    # for baboon in model.baboons.values():
    #     baboon.beta_= np.random.rand(61)
    
    print(f"predicting {i}")
    
    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")

    print(f"new data added {i}")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.extend(score_noniter)
        bc_iterative.extend(score_iter)


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[i, 0], label="noniterative")
    sns.kdeplot(bc_iterative, ax=axes[i, 1], label="iterative")
    axes[i, 0].set_xlim(0, 1)
    axes[i, 1].set_xlim(0, 1)
    axes[i, 0].vlines(np.mean(bc_noniterative), 0, 10, color='red', linestyle='--')
    axes[i, 1].vlines(np.mean(bc_iterative), 0, 10, color='red', linestyle='--')

    # Set titles
    axes[i, 0].set_title(f"Non-iterative (Group {i+1}) Long (all but last 10), mean={np.mean(bc_noniterative):.3f}")
    axes[i, 1].set_title(f"Iterative (Group {i+1}) Long (all but last 10), mean={np.mean(bc_iterative):.3f}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.extend(score_noniter)
        bc_iterative.extend(score_iter)


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[0], label=f"Group {i}, mean={np.mean(bc_noniterative):.3f}")
    sns.kdeplot(bc_iterative, ax=axes[1],  label=f"Group {i}, mean={np.mean(bc_iterative):.3f}")

# Set titles and labels
axes[0].set_title("Non-iterative Long (all but last 10)")
axes[1].set_title("Iterative Long (all but last 10)")
axes[0].set_xlabel("Bray-Curtis Distance")
axes[1].set_xlabel("Bray-Curtis Distance")
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
model = social_model_class_v2.superModel(data_path, metadata_path)
model.alpha_ = np.array([0]*61)
model.gamma_ = 0.95717135
baboon_list = list(model.baboons.values())



mat = np.array([baboon.fit(model.alpha_, model.gamma_)[0] for baboon in baboon_list])

sns.heatmap(pd.DataFrame(mat, columns=baboon.data_I.columns, index=baboon_list ), cmap='viridis')
plt.show()

# Predicting test data

In [5]:
model = model = social_model_class_v2.superModel(data_path, metadata_path)
model.aplpha_ = np.zeros(61)
model.gamma_ = 0.95717135
# cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
# with ProcessPoolExecutor(cpus) as executor:

#     future_to_baboon = {executor.submit(baboon.fit, model.alpha_, model.gamma_): baboon for baboon in model.baboons.values()}

# for future in as_completed(future_to_baboon):
#     baboon = future_to_baboon[future]  # Get the baboon that corresponds to this future
#     baboon.beta_ = future.result()[0]  # Get the result of the computation

/home/jupyter-tomeroron/baboons/social_model_class_v2.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean['collection_date'] = pd.to_datetime(metadata_clean['collection_date'])


In [6]:
model.add_new_data(fr"short_timeseries_data.csv", fr"test_metadata.csv")


/home/jupyter-tomeroron/baboons/social_model_class_v2.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean['collection_date'] = pd.to_datetime(metadata_clean['collection_date'])


In [7]:
test_meta_data = pd.read_csv("test_metadata.csv")
baboons = test_meta_data["baboon_id"].unique()

In [8]:
predictions_iterative = model.predict(baboons, iterative = True)

In [9]:
predictions_iterative 

,g_[Eubacterium]_coprostanoligenes_group,g_[Eubacterium]_hallii_group,g_[Eubacterium]_ruminantium_group,g_Acidaminococcus,g_Alloprevotella,g_Bifidobacterium,g_Butyricicoccus,g_CAG-873,g_Campylobacter,g_Candidatus_Methanogranum,...,g_Ruminococcaceae_UCG-011,g_Ruminococcaceae_UCG-013,g_Ruminococcaceae_UCG-014,g_Ruminococcus_1,g_Senegalimassilia,g_Slackia,g_Solobacterium,g_Streptococcus,g_Succinivibrio,g_Treponema_2
sample_11407-GGTAGCGACGCC-394,0.021741,0.005897,0.002544,0.003043,0.014313,0.207180,0.001673,0.009363,0.002723,0.060134,...,0.000304,0.007643,0.037363,0.008729,0.001152,0.001719,0.003338,0.008504,0.001796,0.003451
sample_11408-AATTTAGGTAGG-395,0.021954,0.005853,0.002738,0.002976,0.014578,0.206484,0.001920,0.009842,0.002593,0.062331,...,0.000337,0.006694,0.035578,0.008507,0.001149,0.001654,0.003358,0.008275,0.001452,0.004199
sample_11408-ATTACGTATCAT-395,0.021709,0.005802,0.002772,0.003013,0.014701,0.207643,0.001964,0.009907,0.002536,0.062811,...,0.000340,0.006332,0.035511,0.008572,0.001153,0.001632,0.003384,0.008035,0.001340,0.004386
sample_11408-TGTCAAAGTGAC-395,0.023370,0.005573,0.003064,0.002383,0.015372,0.179555,0.001955,0.009469,0.002215,0.057896,...,0.000246,0.007369,0.045475,0.011938,0.001066,0.001275,0.003228,0.003568,0.002110,0.007077
sample_11406-ATTATGATTATG-394,0.019708,0.005266,0.004620,0.003551,0.016050,0.199443,0.001982,0.004325,0.001378,0.078131,...,0.000209,0.009372,0.043121,0.013088,0.000888,0.001264,0.002926,0.001876,0.007471,0.005484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_12050-GATGAGTCGCGG-407,0.019366,0.000972,0.000765,0.002789,0.006211,0.285610,0.000610,0.023978,0.000172,0.098358,...,0.002384,0.003645,0.066186,0.020139,0.001099,0.000924,0.004645,0.000631,0.007171,0.039162
sample_12050-CGGGACACCCGA-407,0.023042,0.001962,0.000835,0.003159,0.007326,0.207399,0.000818,0.018998,0.000165,0.094606,...,0.002276,0.005379,0.064203,0.028584,0.000610,0.000533,0.004092,0.000427,0.008455,0.037574
sample_11408-GCAGCGGTGGCT-395,0.016772,0.001419,0.001011,0.003274,0.011344,0.189067,0.000799,0.026510,0.000220,0.099934,...,0.002210,0.003619,0.066089,0.029813,0.000659,0.000580,0.004504,0.002183,0.005978,0.038277
sample_11407-AATAGTCGTGAC-394,0.020940,0.000582,0.000790,0.002904,0.008822,0.199333,0.000923,0.049013,0.000112,0.109534,...,0.004396,0.003816,0.066506,0.022951,0.000903,0.000678,0.005634,0.000295,0.007067,0.040638


In [11]:
predictions_iterative.to_csv("final_model_res.csv")